In [1]:
#source https://www.analyticsvidhya.com/blog/2018/10/mining-online-reviews-topic-modeling-lda/

#import NLP modules
from nltk import FreqDist
import spacy

#modules for cleaning
import string

#imports for data handling
from pathlib import Path
import pandas as pd
pd.set_option("display.max_colwidth", 200)
#import numpy as np

#topic modeling
#import warnings
#disable DeprecationWarnings for gensim
#warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
#warnings.filterwarnings(action='ignore', category=DeprecationWarning)
import gensim
from gensim import corpora

#plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim

C:\Users\kvandoorn\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#load spacy tokenizer
nlp = spacy.load('nl_core_news_sm', disable=['parser', 'ner'])

In [3]:
#define path for data
bestand = Path("social_posts.xlsx")
bestand_insta = Path("insta_posts_all.csv")

In [4]:
#load Dutch and Enlgish stopwords to list
stopbestand = Path(r"C:\Thuiswerken\Python\NL_stopwoorden.txt")
stopwoorden = [line.rstrip('\n') for line in open(stopbestand)]

stopbestand_EN = Path(r"Z:\Datalab\3 Werkwijze\Datasets\Textanalyse\EN_stopwoorden.txt")
stopwoorden_EN = [line.rstrip('\n') for line in open(stopbestand_EN)]

stopwoorden.extend(stopwoorden_EN)

In [5]:
#load file into pandas, select columns, drop empty and doubles
data_clean = pd.read_excel(bestand, usecols=('Hit Sentence','Source','Influencer'))
data_clean.columns = ['text','source','influencer']
data_clean.drop_duplicates(subset='text', inplace=True)
data_clean.dropna(axis=0, how='any', inplace=True)

In [6]:
#load instagram file
data_insta = pd.read_csv(bestand_insta, sep=';', usecols=('Hit Sentence','Source','Influencer'))
data_insta.columns = ['text','source','influencer']
data_insta.drop_duplicates(subset='text', inplace=True)
data_insta.dropna(axis=0, how='any', inplace=True)

In [7]:
#check instagram data
data_insta.head()

,text,source,influencer
0,", groeide het verhaal dat ik samen met de muizen wil vertellen. Een verhaal over creativiteit, spelen en verbinding maken, maar waar ook duurzaamheid, ecologie en respect voor de natuur centraal ...",Instagram,@muizennest
1,"uit 304 RVS. ➡️TIG Gelast. ➡️Zeer hoge en duurzame kwaliteit.\n➡️Perfecte pasvorm, uitwisselbaar met originele downpipe\n➡️Uitvoerig op performance en duurzaamheid getest \nRVSuitlaatdelen.nl is o...",Instagram,@rvsuitlaatdelen
2,gewone bratwurst te onderscheiden. De structuur is ook erg goed. Zeker aan te raden! .\n.\n#sustainableliving #sustainability #duurzaam #duurzaamheid #vegan #vegetarian #savetheplanet #noplanb #no...,Instagram,@hansbertrams
3,Heel veel plezier met je nieuwe Windmill Sophie! Vanaf nu rijdt Blitss officieel dus ook in Delft rond 😃,Instagram,@blitss.eu
4,Het keurmerk voor de duurzame ondernemer⠀⠀⠀⠀⠀⠀⠀⠀⠀\n#duurzaamheid #duurzaam #sustainability #bewustwording #eerlijk #zzp #ondernemer#ondernemer #ondernemerschap\t⠀⠀⠀⠀⠀⠀⠀⠀⠀\n#socialmedia #mkb,Instagram,@thewhitelabelnl


In [8]:
#add instagram data to main file
print(data_clean.shape)
data_clean = data_clean.append(data_insta, ignore_index=True)
print(data_clean.shape)

(42391, 3)
(54016, 3)


In [9]:
#define punctuation and words particular for this case
#make function to clean data

duurzaamwords = ['duurzaam', 'verduurzamen', 'duurzaamheid', 'duurzame', 'qt', 'www', 'com']
punct = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~‘’#'

def cleaning(post):
    post = " ".join(x for x in post.split() if x not in stopwoorden and x not in duurzaamwords and len(x) > 2)
    return post

In [10]:
#clean data: lowercase, remove \n remove url
data_clean['text'] = data_clean.text.str.lower().str.replace(r'\n',' ', regex=False).str.replace(r'http\S+', '', regex=True)
data_clean.head()

,text,source,influencer
0,"de das. aanvullende informatie de deelnemers verzamelen zich bij prinsenhof om samen naar het beginpunt van de wandeling te rijden. omdat het ivn voor duurzaamheid is, wordt het gestimuleerd om m...",Facebook,Inwonersbesteo
1,"betonpalen, 7.203 ton gewicht, 441 middenhuur- en koopappartementen, 4.000 m2 commerciële ruimten, 21 meter beneden nap, sociale duurzaamheid, energieneutraal en innovatie. op het buiten ij, naa...",Facebook,Gebr. van 't Hek Funderingstechnieken
2,", waar je alles kunt vinden mbt tuinontwerp, tuinaanleg en tuinverlichting. maar ook de nieuwste snufjes op het gebied van oa regenwateropvang, duurzaamheid en zonne- energie zijn hier te vinden. ...",Facebook,Intratuin (Almelo)
3,"van ons gemeentehuis. natuurlijke materialen zorgen voor een goede isolatie, maar zijn gewoon ook esthetisch mooi! wil je meer weten? ontdek het -> #lennikrenoveert #wijrenoveren #lennik #gemeen...",Facebook,Gemeente Lennik
4,"aan het werk bent.🥵 let op❗️ enkel te verkrijgen in onze shop nu voor €9 ipv €14 eigenschappen • echte metalen constructie voor duurzaamheid en veiligheid • laag stroomverbruik van 2,5 w • 360...",Facebook,Lalemansolutions


In [11]:
#clean data: remove anything not a letter
data_clean['text'] = data_clean.text.str.replace("[^a-zA-Z]", " ")
data_clean.head(50)

,text,source,influencer
0,de das aanvullende informatie de deelnemers verzamelen zich bij prinsenhof om samen naar het beginpunt van de wandeling te rijden omdat het ivn voor duurzaamheid is wordt het gestimuleerd om m...,Facebook,Inwonersbesteo
1,betonpalen ton gewicht middenhuur en koopappartementen m commerci le ruimten meter beneden nap sociale duurzaamheid energieneutraal en innovatie op het buiten ij naa...,Facebook,Gebr. van 't Hek Funderingstechnieken
2,waar je alles kunt vinden mbt tuinontwerp tuinaanleg en tuinverlichting maar ook de nieuwste snufjes op het gebied van oa regenwateropvang duurzaamheid en zonne energie zijn hier te vinden ...,Facebook,Intratuin (Almelo)
3,van ons gemeentehuis natuurlijke materialen zorgen voor een goede isolatie maar zijn gewoon ook esthetisch mooi wil je meer weten ontdek het lennikrenoveert wijrenoveren lennik gemeen...,Facebook,Gemeente Lennik
4,aan het werk bent let op enkel te verkrijgen in onze shop nu voor ipv eigenschappen echte metalen constructie voor duurzaamheid en veiligheid laag stroomverbruik van w ...,Facebook,Lalemansolutions
5,een tweede leven voor de voormalige twin towers waar akzonobel en stibbe tot kantoor hielden daaraan werkt herontwikkelingsspecialist provast samen met kcap architects planners onder de...,Facebook,Amsterdam Zuidas
6,hebben de opgave dat alle gebouwen in van het aardgas af zijn en door een duurzame warmtebron verwarmd worden tijdens de expeditie duurzame warmte is er gekeken naar opgaven en kan...,Facebook,Gemeente Brielle
7,logistiek jeroen kiest graag voor duurzame oplossingen zelf komt ie op de fiets naar z n werk en ook bij adri zoon probeert hij duurzaamheid te integreren in de dagelijkse praktijk die gloedni...,Facebook,Adri & Zoon Yerseke
8,bouwen afvalscheiding en elektrisch vervoer kunnen zij een belangrijke bijdrage leveren aan de ambities van nieuwegein op het gebied van duurzaamheid daarom ondertekenden wethouder marieke schou...,Facebook,pen.nl
9,design food health en service wellness design en comfort voor lifestyle geori nteerde reizigers met affiniteit voor kunst en cultuur duurzaamheid en work life balance individueel ingericht...,Facebook,TravelTeam


In [12]:
#remove stopwords and particular words
data_clean['text'] = [cleaning(post) for post in data_clean['text']]

In [13]:
data_clean.head(10)

,text,source,influencer
0,das aanvullende informatie deelnemers verzamelen prinsenhof samen beginpunt wandeling rijden ivn gestimuleerd elkaar mee rijden verplicht deelnemers regelen,Facebook,Inwonersbesteo
1,betonpalen ton gewicht middenhuur koopappartementen commerci ruimten meter nap sociale energieneutraal innovatie naast ijburg verrijst straks wooncomplex sluishuis opdracht besix vorm,Facebook,Gebr. van 't Hek Funderingstechnieken
2,vinden mbt tuinontwerp tuinaanleg tuinverlichting nieuwste snufjes gebied regenwateropvang zonne energie vinden neem kijkje maak afspraak collega tuinontwerpstudio kijk,Facebook,Intratuin (Almelo)
3,gemeentehuis natuurlijke materialen zorgen goede isolatie esthetisch mooi weten ontdek lennikrenoveert wijrenoveren lennik gemeentehuis thuisinjegemeente hitte esthetischmooi,Facebook,Gemeente Lennik
4,werk let verkrijgen shop ipv eigenschappen echte metalen constructie veiligheid laag stroomverbruik draaibaar luchtstroom overal naartoe richten usb voeding,Facebook,Lalemansolutions
5,tweede leven voormalige twin towers akzonobel stibbe kantoor hielden daaraan werkt herontwikkelingsspecialist provast samen kcap architects planners naam amsterdam staat,Facebook,Amsterdam Zuidas
6,opgave gebouwen aardgas warmtebron verwarmd expeditie warmte gekeken opgaven kansen eiland verwarmen huizen gebouwen,Facebook,Gemeente Brielle
7,logistiek jeroen kiest graag oplossingen komt fiets werk adri zoon probeert integreren dagelijkse praktijk gloednieuwe volvo sluit naadloos binnenkort vinden nederlandse wegen,Facebook,Adri & Zoon Yerseke
8,bouwen afvalscheiding elektrisch vervoer belangrijke bijdrage leveren ambities nieuwegein gebied ondertekenden wethouder marieke schouten namens gemeente nieuwegein green deal zorgspectrum donderd...,Facebook,pen.nl
9,design food health service wellness design comfort lifestyle geori nteerde reizigers affiniteit kunst cultuur work life balance individueel ingerichte kamers casual local inspired design plek acht...,Facebook,TravelTeam


In [14]:
#function to plot most frequent terms

def freq_words(x, terms=30):
    all_words = ' '.join([text for text in x])
    all_words = all_words.split()
    
    fdist = FreqDist(all_words)
    words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})
    
    #selecting top20 most freq words
    d = words_df.nlargest(columns="count", n = terms)
    plt.figure(figsize=(20,5))
    ax = sns.barplot(data=d, x="word", y="count")
    ax.set(ylabel= 'Count')
    plt.show()

In [15]:
#freq_words(data_clean['text'])

In [16]:
#data_clean['text'] = data_clean['text'].str.replace('#duurzaamheid','')
#freq_words(data_clean['text'])

In [17]:
#convert posts to list
posts = data_clean['text'].tolist()

In [18]:
#tokenize posts
tokenized_posts = pd.Series(posts).apply(lambda x: x.split())
print(tokenized_posts[:1])

0    [das, aanvullende, informatie, deelnemers, verzamelen, prinsenhof, samen, beginpunt, wandeling, rijden, ivn, gestimuleerd, elkaar, mee, rijden, verplicht, deelnemers, regelen]
dtype: object


In [19]:
#function to lemmatize tokens
def lemmatization(texts):
    output = []
    for sent in texts:
        doc = nlp(' '.join(sent))
        output.append([token.lemma_ for token in doc if len(token.lemma_) > 2])
    return output

In [20]:
%%time
#lemmatize tokens
posts_lemma = lemmatization(tokenized_posts)

Wall time: 3min 23s


# topic modeling LDA model

In [21]:
dictionary = corpora.Dictionary(posts_lemma)

In [22]:
doc_term_matrix = [dictionary.doc2bow(post) for post in posts_lemma]

In [23]:
LDA = gensim.models.ldamodel.LdaModel

In [24]:
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=7, random_state=100, chunksize=1000, passes=20)

In [25]:
lda_model.print_topics()

[(0,
  '0.019*"maken" + 0.016*"goed" + 0.013*"gaan" + 0.012*"mens" + 0.010*"vinden" + 0.008*"mee" + 0.008*"komen" + 0.007*"willen" + 0.007*"jaar" + 0.007*"dag"'),
 (1,
  '0.009*"vegan" + 0.008*"sustainable" + 0.006*"together" + 0.006*"organic" + 0.006*"fietsen" + 0.006*"heerlijk" + 0.005*"wassen" + 0.005*"shirt" + 0.005*"solar" + 0.004*"fairtrad"'),
 (2,
  '0.009*"behandelen" + 0.008*"marketing" + 0.007*"fashion" + 0.007*"materiaal" + 0.006*"werkplek" + 0.005*"hoog" + 0.005*"doelstelling" + 0.005*"erin" + 0.005*"kleren" + 0.005*"dringen"'),
 (3,
  '0.009*"komen" + 0.009*"innovatie" + 0.008*"samen" + 0.008*"circulair" + 0.007*"gaan" + 0.007*"ding" + 0.006*"dag" + 0.006*"nieuw" + 0.006*"zien" + 0.005*"energie"'),
 (4,
  '0.018*"bio" + 0.013*"sustainability" + 0.010*"linken" + 0.008*"dag" + 0.008*"link" + 0.008*"sustainableliving" + 0.008*"green" + 0.008*"tips" + 0.007*"sustainable" + 0.007*"inspiratie"'),
 (5,
  '0.010*"utrecht" + 0.009*"nederland" + 0.007*"belegger" + 0.007*"rotterdam" 

In [ ]:
# # Visualize the topics
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
# vis

In [ ]:
# pyLDAvis.save_html(vis, 'topics_metinsta.html')